In [1]:
import os
import cv2

# Directories
data_dir = 'Data/dataforcropping/step 1/'  # Base directory containing images and labels
output_dir = 'Data/dataforcropping/step 2/'  # Base directory to save cropped images

os.makedirs(output_dir, exist_ok=True)

# Helper function to convert normalized bbox to pixel bbox
def convert_bbox(size, box):
    dw = size[0]
    dh = size[1]
    x = box[0] * dw
    y = box[1] * dh
    w = box[2] * dw
    h = box[3] * dh
    x1 = int(x - w / 2)
    y1 = int(y - h / 2)
    x2 = int(x + w / 2)
    y2 = int(y + h / 2)
    return (x1, y1, x2, y2)

# Get list of label files
label_files = [f for f in os.listdir(data_dir) if f.endswith('.txt') and 'classes.txt' not in f]

for label_file in label_files:
    # Read the label file
    with open(os.path.join(data_dir, label_file), 'r') as file:
        lines = file.readlines()

    # Read the corresponding image
    image_file = label_file.replace('.txt', '.jpg')
    image_path = os.path.join(data_dir, image_file)
    if not os.path.exists(image_path):
        print(f"Image file {image_file} not found, skipping.")
        continue

    image = cv2.imread(image_path)
    img_height, img_width = image.shape[:2]

    # Process each bounding box in the label file
    for line in lines:
        parts = line.strip().split()
        class_id = parts[0]
        bbox = list(map(float, parts[1:]))

        # Convert normalized bbox to pixel bbox
        x1, y1, x2, y2 = convert_bbox((img_width, img_height), bbox)

        if class_id == 20:
            x1 += 120
            x2 += 120
        
        # Crop the image
        cropped_image = image[y1:y2, x1:x2]

        # Save the cropped image to the corresponding class directory
        class_dir = os.path.join(output_dir, class_id)
        os.makedirs(class_dir, exist_ok=True)
        cropped_image_filename = f"{os.path.splitext(image_file)[0]}_{x1}_{y1}_{x2}_{y2}.jpg"
        cropped_image_path = os.path.join(class_dir, cropped_image_filename)
        cv2.imwrite(cropped_image_path, cropped_image)

print("Cropping completed.")


Cropping completed.


In [3]:
import os
import cv2
import numpy as np

cropped_image_dir = 'Data/dataforcropping/step 2/' 
converted_img_dir = 'Data/dataforcropping/step 3/' 



desired_size = 640  # Example size, adjust as needed

os.makedirs(converted_img_dir, exist_ok=True)

def resize_and_pad(image, desired_size):
    old_size = image.shape[:2]  # old_size is in (height, width) format
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    
    # Resize the image
    resized_image = cv2.resize(image, (new_size[1], new_size[0]))
    
    # Pad the image to the desired size
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    
    color = [0, 0, 0]  # Padding color (black)
    new_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    
    return new_image

for root, dirs, files in os.walk(cropped_image_dir):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = cv2.imread(image_path)
            
            if image is None:
                continue
            
            resized_image = resize_and_pad(image, desired_size)
            
            # Create corresponding output directory
            relative_path = os.path.relpath(root, cropped_image_dir)
            output_subdir = os.path.join(converted_img_dir, relative_path)
            os.makedirs(output_subdir, exist_ok=True)
            
            output_path = os.path.join(output_subdir, file)
            cv2.imwrite(output_path, resized_image)

print("Resizing and padding completed.")


Resizing and padding completed.
